In [19]:
# (미니배치)확률 경사하강법을 위해 필요한 
from sklearn.utils import shuffle

import numpy as np
import tensorflow as tf

In [20]:
# 변수 정의해두기
M = 2 #입력데이터 차원
K = 3 #클래스 수
n = 100 #각 클래스에 있는 데이터 수
N = n*K #전체 데이터 수

In [21]:
# 샘플데이터 생성
X1 = np.random.randn(n, M) + np.array([0, 10])
X2 = np.random.randn(n, M) + np.array([5, 5])
X3 = np.random.randn(n, M) + np.array([10, 0])

Y1 = np.array([[1, 0, 0] for i in range(n)])
Y2 = np.array([[0, 1, 0] for i in range(n)])
Y3 = np.array([[0, 0, 1] for i in range(n)])

X = np.concatenate((X1, X2, X3), axis=0)
Y = np.concatenate((Y1, Y2, Y3), axis=0)

In [22]:
# 이진분류에서 Sigmoid 였던 부분을 Softmax로 변형만 하면 다중클래스로 분류하는 것 출력 가능
W = tf.Variable(tf.zeros([M, K]))
b = tf.Variable(tf.zeros([K]))

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

x = tf.placeholder(tf.float32, shape=[None, M])
t = tf.placeholder(tf.float32, shape=[None, K])
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [23]:
# 오차함수 정의해주기 (Cross_entropy)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(t*tf.log(y), reduction_indices=[1])) # reduction_indices는 행렬의 합 방향을 정해주는 것

In [24]:
# 경사하강법~
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [25]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1)) # 예측값과 결과값이 맞는지 확인하는 것

In [26]:
batch_size = 50 # 미니배치 크기
n_batches = N //batch_size

In [27]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [29]:
# 확률경사하강에서는 각 에폭마다 데이터를 섞으며 실제로 학습을 수행하는 코드는 아래와 같음
for epoch in range(20):
    X_, Y_ = shuffle(X, Y)
    
    for i in range(n_batches):
        start = i*batch_size
        end = start + batch_size
        
        sess.run(train_step, feed_dict={
            x:X_[start:end],
            t:Y_[start:end]
        })

In [30]:
# 결과 확인해보기
X_, Y_ = shuffle(X, Y)

classified = correct_prediction.eval(session=sess, feed_dict={
    x:X_[0:10], t:Y_[0:10]
})
prob = y.eval(session=sess, feed_dict={
    x:X_[0:10]
})

print('Classified')
print(classified)
print()
print('Output Probability')
print(prob)

Classified
[ True  True  True  True  True  True  True  True  True  True]

Output Probability
[[7.0767275e-10 1.5523518e-03 9.9844766e-01]
 [4.5982271e-09 7.8458088e-03 9.9215418e-01]
 [9.9729162e-01 2.7083482e-03 7.5318818e-10]
 [9.9565142e-01 4.3485877e-03 2.7380603e-08]
 [2.5889883e-03 9.6762854e-01 2.9782459e-02]
 [1.1636893e-09 2.9577001e-03 9.9704224e-01]
 [1.5350128e-07 1.1909149e-02 9.8809063e-01]
 [6.9761185e-08 1.9177491e-02 9.8082238e-01]
 [9.9973887e-01 2.6110641e-04 1.4702017e-10]
 [3.3475664e-09 5.0440622e-03 9.9495596e-01]]


### 이번에는 또 Keras로 구현해보기

In [31]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

In [33]:
model = Sequential()
model.add(Dense(input_dim=M, units=K))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.1))

In [34]:
minibatch_size = 50
model.fit(X, Y, epochs=20, batch_size=minibatch_size)

Train on 300 samples
Epoch 1/20
300/300 [==============================] - 1s 4ms/sample - loss: 0.6177
Epoch 2/20
300/300 [==============================] - 0s 57us/sample - loss: 0.0508
Epoch 3/20
300/300 [==============================] - 0s 63us/sample - loss: 0.0458
Epoch 4/20
300/300 [==============================] - 0s 67us/sample - loss: 0.0432
Epoch 5/20
300/300 [==============================] - 0s 60us/sample - loss: 0.0405
Epoch 6/20
300/300 [==============================] - 0s 60us/sample - loss: 0.0386
Epoch 7/20
300/300 [==============================] - 0s 57us/sample - loss: 0.0364
Epoch 8/20
300/300 [==============================] - 0s 60us/sample - loss: 0.0346
Epoch 9/20
300/300 [==============================] - 0s 60us/sample - loss: 0.0332
Epoch 10/20
300/300 [==============================] - 0s 63us/sample - loss: 0.0318
Epoch 11/20
300/300 [==============================] - 0s 60us/sample - loss: 0.0309
Epoch 12/20
300/300 [==============================] -

In [35]:
X_, Y_ = shuffle(X, Y)
classes = model.predict(X_[0:10], batch_size=minibatch_size)
classes

C:\Users\Main\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


array([[1.2512995e-02, 9.8428059e-01, 3.2064163e-03],
       [9.9778509e-01, 2.2148939e-03, 1.1563247e-09],
       [1.4187043e-09, 5.3567705e-03, 9.9464321e-01],
       [9.9543446e-01, 4.5655523e-03, 1.9934296e-09],
       [1.4097329e-02, 9.8362768e-01, 2.2750632e-03],
       [4.5600537e-11, 5.1459153e-03, 9.9485415e-01],
       [4.4743448e-08, 3.6489367e-02, 9.6351057e-01],
       [9.6245505e-02, 9.0240645e-01, 1.3479975e-03],
       [9.9745041e-01, 2.5495416e-03, 1.6806589e-10],
       [1.2363078e-09, 5.0999224e-03, 9.9490005e-01]], dtype=float32)